# Calibrating WFI Exposures with RomanCal 

***

## Kernel Information and Read-Only Status

To run this notebook, please select the "Roman Calibration" kernel at the top right of your window.

This notebook is read-only. You can run cells and make edits, but you must save changes to a different location. We recommend saving the notebook within your home directory, or to a new folder within your home (e.g. <span style="font-variant:small-caps;">file > save notebook as > my-nbs/nb.ipynb</span>). Note that a directory must exist before you attempt to add a notebook to it.

## Imports
 Libraries used
- *romancal* for running the processing pipeline
- *roman_datamodels* for opening Roman WFI ASDF files
- *asdf* for opening Roman WFI ASDF files
- *os* for checking if files exist
- *s3fs* for streaming files from an S3 bucket

In [ ]:
import roman_datamodels as rdm
import asdf
import romancal
from romancal.pipeline import ExposurePipeline
import os
import s3fs

## Introduction
The purpose of this notebook is to calibrate Level 1 (L1; uncalibrated ramp cube) data with the Roman WFI science calibration pipeline RomanCal (Python package name `romancal`) to produce Level 2 (L2; calibrated rate image) exposure level data. To learn more, please visit the [RDox pages on the Exposure Level Pipeline](https://roman-docs.stsci.edu/data-handbook-home/roman-stsci-data-pipelines/exposure-level-pipeline).

Details about the Roman data levels can be found in the RDox article [Data Levels and Products](https://roman-docs.stsci.edu/data-handbook-home/wfi-data-format/data-levels-and-products). A L1 file contains a single uncalibrated ramp in units of Data Numbers (DN).  L1 files are three-dimensional data cubes, one dimension for time and two dimensions for image coordinates, that are shaped as  arrays with (N resultants, 4096 image rows, 4096 image columns). L2 WFI files are calibrated rate images in instrumental units of DN / second.  They are two-dimensional arrays shaped as (4088 image rows, 4088 image columns).

***

## Tutorial Data
In this tutorial, we use L1 WFI data files simulated with `romanisim`. We will use as an example the output from running the [Roman I-Sim](../romanisim/romanisim.ipynb) tutorial notebook. If you did not run the simulation tutorial, then the files are also stored in the RRN S3 bucket. For more information on how to access these data, see the [Data Discovery and Access](../data_discovery_and_access/data_discovery_and_access.ipynb) tutorial.

## Run romancal on L1 Data
To run `romancal` on the L1 data, there are two options:
1. You can use the exposure-level pipeline to run all steps (basic), or
2. You can run one or more individual steps (advanced).

### Basic Example: Full Pipeline

The input file for our example is a WFI L1 ASDF file. We must copy the file to our local storage as the RomanCal exposure pipeline currently only works with an input file on disk. Individual pipeline steps (described further below) can run directly on datamodels in memory.

In [ ]:
if not os.path.exists('r0003201001001001004_0001_wfi01_f106_uncal.asdf'):
    asdf_dir_uri = 's3://roman-sci-test-data-prod-summer-beta-test/'
    fs = s3fs.S3FileSystem()

    asdf_file_uri = asdf_dir_uri + 'AAS_WORKSHOP/r0003201001001001004_0001_wfi01_f106_uncal.asdf'
    _ = fs.get(asdf_file_uri, '.')

Let's start with the basic example of running the "complete" pipeline. Please note that we are skipping the final two steps in the pipeline: source catalog and tweakreg (see note below). Alignment to Gaia is not necessary for these data as they are simulated using the Gaia catalog. There are other optional parameters that may be set for individual steps in a similar manner, and more information can be found in the [romancal documentation](https://roman-pipeline.readthedocs.io/en/latest/index.html).

The save_results optional parameter will save the resulting file to your local disk. You can enable this by setting the value to `True`. In our example, we will keep the output calibrated result in memory without saving it locally.

**Note:** Recent testing has shown that the source catalog step may crash under specific conditions. Caution is advised while we work on fixing the problem.

In [ ]:
result = ExposurePipeline.call('r0003201001001001004_0001_wfi01_f106_uncal.asdf', save_results=False, steps={'source_catalog': {'skip': True}, 'tweakreg': {'skip': True}})

The output from the exposure pipeline is currently returned as a list. You will need to index the results variable with a `[0]` to access the datamodel object.

In [ ]:
type(result)

In [ ]:
type(result[0])

In the example above, you can see how we passed optional parameters to individual steps in the pipeline. The pipeline steps in order are:

- `romancal.dq_init.dq_init_step`: Bad pixel masking and data quality initialization
- `romancal.saturation.SaturationStep`: Saturation flagging up-the-ramp
- `romancal.refpix.RefPixStep`: 1/f noise correction
- `romancal.linearity.LinearityStep`: Classic non-linearity correction
- `romancal.dark_current.DarkCurrentStep`: Dark current subtraction
- `romancal.ramp_fitting.ramp_fit_step`: Jump detection and fitting up-the-ramp
- `romancal.assign_wcs.AssignWcsStep`: Initialize the WCS with the pointing information
- `romancal.flatfield.FlatFieldStep`: Apply the flat field to the data
- `romancal.photom.PhotomStep`: Populate photometric calibration information
- `romancal.source_detection.SourceCatalog`: Run source detection on the image, perform point spread function (PSF) fitting photometry, and generate a source catalog
- `romancal.tweakreg.TweakRegStep`: Match sources to Gaia and update WCS information

Note that the ramp fitting step transforms the datamodel in memory. Therefore, steps following ramp fitting cannot be applied to a data model that has not undergone ramp fitting, and similarly, steps preceeding ramp fitting should not be applied to a  data model after this step.

### Advanced Example: Running Individual Pipeline Steps

Now, for a more advanced use case, let's update the WCS based on the pointing information. In this example, let's imagine that we have simulated a field in a L2 calibrated file, and we want to update the distortion model in the file's GWCS (Generalized World Coordinate System) object. We can accomplish this by running the individual AssignWcsStep on an L2 ASDF file. If you ran the previous cells, the file should already be saved on disk. If not, we can stream it from the S3 bucket.

In [ ]:
if os.path.exists('r0003201001001001004_0001_wfi01_f106_cal.asdf'):
    dm = rdm.open(f)
else:
    asdf_dir_uri = 's3://roman-sci-test-data-prod-summer-beta-test/'
    fs = s3fs.S3FileSystem()

    asdf_file_uri = asdf_dir_uri + 'AAS_WORKSHOP/r0003201001001001004_0001_wfi01_f106_cal.asdf'
    with fs.open(asdf_file_uri, 'rb') as f:
        af = asdf.open(f)
        dm = rdm.open(af)

Next, let's run AssignWcsStep on the datamodel. Doing so will return an updated datamodel in memory:

In [ ]:
dm.info()

In [ ]:
result = romancal.assign_wcs.AssignWcsStep.call(dm)

In this case, the output from the step is the datamodel rather than a list:

In [ ]:
type(result)

If we would like to use our own version of the distortion reference file rather than the one from CRDS, then we can use the file with the override_distortion optional parameter:

In [ ]:
# wcs_step = romancal.assign_wcs.AssignWcsStep.call(dm, override_distortion='my_distortion_file.asdf')

Similar override parameters exist for all reference file types. More information on WFI reference file types may be found in the RDox article [CRDS for Reference Files](https://roman-docs.stsci.edu/data-handbook-home/accessing-wfi-data/crds-for-reference-files).

As before, we directed the updated datamodel to the variable "result" in active memory. We can access the information in the "result" (such as the gwcs object or data array) as in any other datamodel object. To write the result to disk, we can do so with the `save()` method. We can also pass this datamodel along to the next pipeline step and chain steps together. For more information on working with datamodels, see the [Working with ASDF](../working_with_asdf/working_with_asdf.ipynb) tutorial.

## Additional Resources
- [romanisim](https://romanisim.readthedocs.io/en/latest/index.html)
- [romancal](https://roman-pipeline.readthedocs.io/en/latest/index.html)
- [Roman Documentation](https://roman-docs.stsci.edu)

## About this Notebook
**Author:** Sanjib Sharma, Tyler Desjardins  
**Updated On:** 2025-01-10

***

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 